In [7]:
import pandas as pd, json, tabula, jsonlines, logging,time, requests, re
from typing import *
# Configure logging
logging.basicConfig(filename='log.txt', level=logging.INFO)


def json_to_df(data:list):
    return pd.DataFrame(data)


def read_json(INPUT_PATH:str, file_name:str):
    # read the file path
    #change to the dict that works
    with open(f'{INPUT_PATH}/{file_name}.json') as json_file:
        data = json.load(json_file)
        return data
    
def merge_lists(file_path: str, json_list: str) -> List[Any]:
    #This function merges an infinite number of all the lists 
    #in range of data source
    
    merged_list = []
    for json_data in json_list:
        merged_list.append(pd.DataFrame(read_json(json_data)))
    return merged_list


def read_jsonline (FILEPATH: str) -> List [Dict [str, Any]]:

    # Reads a JSON file and returns a list of dictionaries with the student details
    #using json library

    file_details = []
    # your code goes here
    jsonl_files_list=['json1','json2']
   

    # Open the file and read lines
    for file_name in jsonl_files_list:
        with open(f"{FILEPATH}/{file_name}.jsonl", 'r') as file:
            # Iterate through each line in a .jsonl file
            for line in file:
                # Load JSON from the line
                data = json.loads(line)

                # Now 'data' contains the content of the line as a Python dictionary
                # You can process it as needed
    #             print(data)
                file_details.append(data)
    return file_details


def read_jsonl(FILEPATH: str,file_name:str):
    
      # Reads a JSONL file and returns a list of dictionaries with the student details
    #using jsonlines library
    with jsonlines.open(f'{PATH}/{file_name}.jsonl') as reader:
        for obj in reader:
            print(obj)

# this function writes to a json file            
def write_to_json(OUTPUT_PATH: str,file_name:str,data):
    
      # writes to a JSON file using json library
    with open(f'{OUTPUT_PATH}/{file_name}.json', 'w') as json_file:
        json.dump(data, json_file, indent=None)
        
       # Log a message with a timestamp
    timestamp = time.ctime()
    log_message = f"{timestamp} - File {OUTPUT_PATH}/{file_name}.json written to successfully"
    logging.info(log_message)


def write_to_jsonl(FILEPATH: str,file_name:str,data):
    
      # writes to a JSONL file using jsonlines library
    with jsonlines.open(f'{PATH}/{file_name}.jsonl', mode='a') as writer:
        writer.write(data)

       # Log a message with a timestamp
    timestamp = time.ctime()
    log_message = f"{timestamp} - File {FILEPATH}/{file_name}.jsonl written to successfully"
    logging.info(log_message)


def read_pdf(FILEPATH:str,file_name:str)->List:
    curr_pages = tabula.read_pdf(f"{FILEPATH}/{file_name}", pages='all',multiple_tables=True)
    return curr_pages


def convert_pdf_to_json(tables:list,output_loc:str)->None:
    # Process and organize the data into a JSON format
    json_data = []
    for table in tables:
        # Assuming each table is a list of dictionaries
        table_data = table.to_dict(orient='records')
        json_data.append(table_data)

    # Write the JSON data to a file
    with open(output_loc, 'w') as json_file:
        json.dump(json_data, json_file, indent=None)
        
# this func removes html tags from a string 
def remove_html_tags(text):
    """Remove html tags from a string"""
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

# this function retrieves the etymology of a word using the wordnik api        
def getEtymology(word_to_search:str)->str:
    
    word=word_to_search.strip().lower()
    url=f'https://api.wordnik.com/v4/word.json/{word}/etymologies'

    params={
        'useCanonical':'false',
        'api_key':'YOUR_WORDNIK_API_KEY'
    }
    response=requests.get(url,params=params)
    
    try:
        return remove_html_tags(response.json()[0])
    except KeyError:
        return response.json()['message']
